<a href="https://colab.research.google.com/github/alaa-1104/Driver-Drowsiness-Detection/blob/main/drowsiness_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import cv2
import dlib
from scipy.spatial import distance
from google.colab.patches import cv2_imshow

def calculate_EAR(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

video_path = "/content/drive/MyDrive/Driver-Drowsiness-Detection/demo/eye_blinking.mp4"
model_path = "/content/drive/MyDrive/Driver-Drowsiness-Detection/shape_predictor_68_face_landmarks.dat"

cap = cv2.VideoCapture(video_path)
hog_face_detector = dlib.get_frontal_face_detector()
dlib_facelandmark = dlib.shape_predictor(model_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = hog_face_detector(gray)

    for face in faces:
        landmarks = dlib_facelandmark(gray, face)
        leftEye, rightEye = [], []

        for n in range(36, 42):
            x, y = landmarks.part(n).x, landmarks.part(n).y
            leftEye.append((x, y))
            cv2.circle(frame, (x, y), 1, (0,255,0), -1)

        for n in range(42, 48):
            x, y = landmarks.part(n).x, landmarks.part(n).y
            rightEye.append((x, y))
            cv2.circle(frame, (x, y), 1, (0,255,0), -1)

        left_ear = calculate_EAR(leftEye)
        right_ear = calculate_EAR(rightEye)
        EAR = round((left_ear + right_ear) / 2, 2)

        if EAR < 0.26:
            cv2.putText(frame, "DROWSY", (20,100),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 3)

        cv2.putText(frame, f"EAR: {EAR}", (20,50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)

    cv2_imshow(frame)

cap.release()
